In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 9


True

In [4]:
api_key = os.getenv('PINECONE_API_KEY')

In [6]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [7]:
from pinecone import Pinecone , ServerlessSpec

index_name = "hybrid-search-langchain"

pc = Pinecone(api_key = api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index( name = index_name , dimension= 384 , metric= 'dotproduct' , spec = ServerlessSpec(cloud = 'aws' , region = 'us-east-1') )

In [9]:
index = pc.Index(index_name)
index

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

/home/shreyans-jain/Desktop/ML/LANGCHAIN-DEMO/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to /home/shreyans-
[nltk_data]     jain/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/shreyans-
[nltk_data]     jain/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
sentences = [
    "In 2024 , I finished my masters education in computer science",
    "In 2016 , I finished my bachelors education in computer science",
    "In 2020 , I started my first job as a software engineer"
]

In [16]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/shreyans-
[nltk_data]    |     jain/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /home/shreyans-
[nltk_data]    |     jain/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/shreyans-jain/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/shreyans-jain/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/shreyans-jain/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    

True

In [18]:
bm25_encoder.fit(sentences)

bm25_encoder.dump("bm25_values.json")

bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 5079.90it/s]


In [20]:
retriever = PineconeHybridSearchRetriever(embeddings = embeddings , sparse_encoder = bm25_encoder , index = index)

In [21]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x743c24331b50>, index=<pinecone.data.index.Index object at 0x743d5406e750>)

In [22]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:01<00:00,  1.89s/it]


In [26]:
retriever.invoke("In which year I started my first job?")

[Document(page_content='In 2020 , I started my first job as a software engineer'),
 Document(page_content='In 2024 , I finished my masters education in computer science'),
 Document(page_content='In 2016 , I finished my bachelors education in computer science')]